# 0. Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import json
import tqdm

from rdkit import Chem
from tqdm import tqdm
from thermo import functional_groups
from Bio import Entrez
from chembl_structure_pipeline import checker

from rdkit.Chem import rdMolDescriptors, Descriptors, Lipinski, Crippen, inchi
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from concurrent.futures import ThreadPoolExecutor, as_completed

from data_gathering import download_and_save

[05:05:20] Initializing Normalizer


In [2]:
data_folder = 'data/AID488997' # Name your data folder
dataset_name = 'AID488997_choline_transporter' # Name your dataset
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# 1. Data gathering

Before importing data, need to identify which AIDs will be included. Data will be imported from https://pubchem.ncbi.nlm.nih.gov/assay/pcget. For more information on PubChem's programmatic access, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/bioassays. Some other programmatic access options available such as PUG-REST. However, these might not be optimal for bulk retrieval or handling of large dataset due to the limitation of request volume. 

Data for individual assays include 7 required columns (CIDs, isomeric SMILES, etc.) and optional test results. Refer to https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/CSV/README for further details. For datasets intended for regression model, additional columns could be extracted accordingly.

In [3]:
# Desired AIDs:
AIDs = [488975, 504840, 493221, 588401, 602208, 493222]

#Keep unique values in list AIDs (since there could be overlapping AIDs from different targets or project)
AIDs = list(set(AIDs))
AIDs = [str(AID) for AID in AIDs]
print('Number of datasets retrieving: ', len(AIDs))

Number of datasets retrieving:  6


In [4]:
#Data to be extracted from the assay:
col_list = ['PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME']
download_and_save(AIDs, data_folder, col_list)

1 out of 6 complete
2 out of 6 complete
3 out of 6 complete
4 out of 6 complete
5 out of 6 complete
6 out of 6 complete


# 2. Isomeric SMILES

For the purpose of our project, we would like to include isomeric form of SMILES representation in our final dataset. Although PubChem claimed that their datatable should include isomeric SMILES (https://pubchem.ncbi.nlm.nih.gov/docs/bioassays), some dataset might include non-isomeric SMILES. This step is to import isomeric SMILES based on CIDs. 

Several packages such as RDkit have modules to return isomeric SMILES from a given input SMILES. However, for consistency, we decided to use the PubChem Identifier Exchange Service, which take an input identifier (CIDs, SMILES, InChI, etc.)  and return the corresponding identifier (CIDs, isomeric SMILES, InChIs, etc.). Here, we export the list of CIDs for compounds in our dataset and use this server to retrieve their isomeric SMILES. For more information, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service

In [5]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_2'):
    os.makedirs(f'{data_folder}/before_finished/step_2')

#Export list of CIDs to csv with one column without the column name:
for AID in AIDs:
    assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
    assay['PUBCHEM_CID'].to_csv(f'{data_folder}/before_finished/step_2/CID{AID}.csv', index=False, header=False)
    
"""
After this step, submit the lists of CIDs at https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi 
    Operator type: "same CID" 
    Output IDs "SMILES" (isomeric SMILES by default) 
    Output method: "Two column file showing each input output-correspondence"
    Compression: "No compression"
Refer to https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service for more details.
The output files (converted isomeric SMILES) should be named as "isomeric_smi_{AID}.txt" and saved to the "step_2" folder.
"""

'\nAfter this step, submit the lists of CIDs at https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi \n    Operator type: "same CID" \n    Output IDs "SMILES" (isomeric SMILES by default) \n    Output method: "Two column file showing each input output-correspondence"\n    Compression: "No compression"\nRefer to https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service for more details.\nThe output files (converted isomeric SMILES) should be named as "isomeric_smi_{AID}.txt" and saved to the "step_2" folder.\n'

In [6]:
def check_isomeric_smiles(AIDs):
    """
    Check if the SMILES in the assay are isomeric or not.
    Input: AIDs (list of strings)
    Output: non_isomeric_smi_cids (dictionary with AID as key and list of non-isomeric CIDs as values for the datasets in AIDs
    """
    non_isomeric_smi_cids = {}
    for AID in AIDs:    
        non_isomeric_smi_cids[AID] = []
        #import SMILES.txt file as a table:
        correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
        assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')

        #compare smiles in assay with smiles in correct_smiles:
        for cid in assay['PUBCHEM_CID']:
            if assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'].values[0] != correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]:
                non_isomeric_smi_cids[AID].append(cid)

        if len(non_isomeric_smi_cids[AID]) == 0:
            print(f'All SMILES in AID {AID} are isomeric')
        else:
            print(f'There are some non-isomeric SMILES in AID {AID}:')
            print(non_isomeric_smi_cids[AID])

    return non_isomeric_smi_cids

def update_isomeric(AIDs, non_isomeric_smi_cids):
    """
    Update the SMILES in the assay to isomeric SMILES.
    Input: AIDs (list of strings), non_isomeric_smi_cids (dictionary with AID as key and list of non-isomeric CIDs as values)
    """
    with open(f'{data_folder}/before_finished/step_2/non_isomeric_smi_cids.txt', 'w') as f:
        # record the non-isomeric SMILES 
        for AID in AIDs:
            assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
            correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
            f.write(f'AID {AID}: {non_isomeric_smi_cids[AID]}\n')

            for cid in non_isomeric_smi_cids[AID]:
                f.write(f'CID {cid}: {assay.loc[assay["PUBCHEM_CID"] == cid, "PUBCHEM_EXT_DATASOURCE_SMILES"].values[0]} -> {correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]}\n')
                assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'] = correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]

            f.write(f'===\n')
            assay.to_csv(f'{data_folder}/before_finished/step_2/AID{AID}.csv', index=False)       

In [7]:
non_isomeric_smi_cids = check_isomeric_smiles(AIDs)

All SMILES in AID 602208 are isomeric
All SMILES in AID 493221 are isomeric
All SMILES in AID 493222 are isomeric
All SMILES in AID 504840 are isomeric
There are some non-isomeric SMILES in AID 488975:
[1952060, 1979247, 3640026, 3730758, 3330059, 1829082, 3593962, 2997662, 2999888, 2997957]
All SMILES in AID 588401 are isomeric


Note: Here they returned that three smiles in AID449739 were not isomeric. This shows that the SMILES representation of some compounds in the given datasets might not be isomeric.

In [8]:
update_isomeric(AIDs, non_isomeric_smi_cids)

# 3. InChI

We would like to include standard InChI to diversify users' choice of which data they would like to use for their own benchmark. 

In [9]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_3'):
    os.makedirs(f'{data_folder}/before_finished/step_3')

Again, it is convenient to use the PubChem Identifier Exchange Service (https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi) with operator type "same CID" and Output IDs "InChI" to retrieve InChI from a given list of input CIDs. The same CID lists from STEP 2 could be used here. The resulted InChIs could be checked if being standard by indentifying the presence of 'InChI=1S' at the begining of each InChI string.

In [10]:
"""
CID lists (in "step_2" folder should be submitted to PubChem Identifier Exchange Service)
    Operator type: "same CID" 
    Output IDs "InChI"
    Output method: "Two column file showing each input output-correspondence"
    Compression: "No compression"
InChI list should be saved into "step_3" folder, named as "std_inchi_{AID}.txt" 
"""
# Import dataframes:
for AID in AIDs: 
    exec(f'AID{AID} = pd.read_csv("{data_folder}/before_finished/step_2/AID{AID}.csv")')
    exec(f'AID{AID}_InChI = pd.read_csv("{data_folder}/before_finished/step_3/std_inchi_{AID}.txt", sep="\\t", header=None)')

In [11]:
#Check if they are all standard InChI:
for AID in AIDs:
    check_inchi = f"""
non_standard_InChI = []
for i in range(len(AID{AID}_InChI[1])):
    if not AID{AID}_InChI[1][i].startswith('InChI=1S'):
        non_standard_InChI.append(AID{AID}_InChI[1][i])
if not non_standard_InChI:
    print('All InChI in AID{AID} are standard')
else:
    print('There are some non-standard InChI in AID{AID}')
    print(non_standard_InChI)
    print('===')
"""
    exec(check_inchi)

All InChI in AID602208 are standard
All InChI in AID493221 are standard
All InChI in AID493222 are standard
All InChI in AID504840 are standard
All InChI in AID488975 are standard
All InChI in AID588401 are standard


Now we concatenate the InChIs in our tables:

In [12]:
# Update and save the files
for AID in AIDs: 
    update_inchi = f"""
AID{AID}_InChI_dict = dict(zip(AID{AID}_InChI[0], AID{AID}_InChI[1]))
AID{AID}['InChI'] = AID{AID}['PUBCHEM_CID'].map(AID{AID}_InChI_dict)
AID{AID}['InChI'] = AID{AID}['InChI'].astype(str)
AID{AID}.to_csv(r"{data_folder}/before_finished/step_3/AID{AID}.csv", index=False)
"""
    exec(update_inchi)

# 4. Check duplicates

When checking duplicates in the datasets, we would like to know if there are
1) Multiple identical molecules
2) Molecules with identical CID but different InChIs or SMILES
3) Molecules with identical InChI but with different CIDs or SMILES

In [13]:
#import:
for AID in AIDs:
    exec(f"AID{AID} = pd.read_csv(r'{data_folder}/before_finished/step_3/AID{AID}.csv', sep=',', header=0)")

In [14]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_4'):
    os.makedirs(f'{data_folder}/before_finished/step_4')

## 4.1. Checking identical molecules

In [15]:
#Return all duplicates by comparing InChI, SMILES, and CIDs:
for AID in AIDs:
    check_duplicate = f"""
AID{AID}_duplicates_InChI = AID{AID}[AID{AID}.duplicated(subset=['InChI'], keep=False)]
AID{AID}_duplicates_SMILES = AID{AID}[AID{AID}.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]
AID{AID}_duplicates_CIDs = AID{AID}[AID{AID}.duplicated(subset=['PUBCHEM_CID'], keep=False)]
print('Number of AID{AID} InChI duplicates: ', len(AID{AID}_duplicates_InChI))
print('Number of AID{AID} SMILES duplicates: ', len(AID{AID}_duplicates_SMILES))
print('Number of AID{AID} CID duplicates: ', len(AID{AID}_duplicates_CIDs))
"""
    exec(check_duplicate)

Number of AID602208 InChI duplicates:  0
Number of AID602208 SMILES duplicates:  0
Number of AID602208 CID duplicates:  0
Number of AID493221 InChI duplicates:  0
Number of AID493221 SMILES duplicates:  0
Number of AID493221 CID duplicates:  0
Number of AID493222 InChI duplicates:  0
Number of AID493222 SMILES duplicates:  0
Number of AID493222 CID duplicates:  0
Number of AID504840 InChI duplicates:  0
Number of AID504840 SMILES duplicates:  0
Number of AID504840 CID duplicates:  0
Number of AID488975 InChI duplicates:  237
Number of AID488975 SMILES duplicates:  185
Number of AID488975 CID duplicates:  183
Number of AID588401 InChI duplicates:  0
Number of AID588401 SMILES duplicates:  0
Number of AID588401 CID duplicates:  0


In [16]:
#write duplicates to a txt file: 
with open(f'{data_folder}/before_finished/step_4/duplicates.txt', 'w') as f:
    for AID in AIDs: 
        duplicates_InChI = eval(f'AID{AID}_duplicates_InChI')
        duplicates_SMILES = eval(f'AID{AID}_duplicates_SMILES')
        duplicates_CIDs = eval(f'AID{AID}_duplicates_CIDs')
        f.write(f'\n\nAID{AID} InChI duplicates:\n')
        f.write(duplicates_InChI.to_string())
        f.write(f'\nAID{AID} SMILES duplicates:\n')
        f.write(duplicates_SMILES.to_string())
        f.write(f'\nAID{AID} CID duplicates:\n')
        f.write(duplicates_CIDs.to_string())

## 4.2. Same CIDs but different chemical representations

In [17]:
#reindex
for AID in AIDs: 
    exec(f"AID{AID}_duplicates_CIDs.reset_index(drop=True, inplace=True)")

In [18]:
with open(f'{data_folder}/before_finished/step_4/sameCID_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameCID_differentInChI = []
        sameCID_differentSMILES = []
        duplicates_CIDs = eval(f'AID{AID}_duplicates_CIDs')
        for i in range(len(duplicates_CIDs['PUBCHEM_CID'])):
            for j in range(i+1, len(duplicates_CIDs['PUBCHEM_CID'])):
                if duplicates_CIDs['PUBCHEM_CID'][i] == duplicates_CIDs['PUBCHEM_CID'][j]:
                    if duplicates_CIDs['InChI'][i] != duplicates_CIDs['InChI'][j]:
                        sameCID_differentInChI.append((duplicates_CIDs['PUBCHEM_CID'][i], duplicates_CIDs['PUBCHEM_CID'][j]))
                    if duplicates_CIDs['PUBCHEM_EXT_DATASOURCE_SMILES'][i] != duplicates_CIDs['PUBCHEM_EXT_DATASOURCE_SMILES'][j]:
                        sameCID_differentSMILES.append((duplicates_CIDs['PUBCHEM_CID'][i], duplicates_CIDs['PUBCHEM_CID'][j]))

        if sameCID_differentInChI == []:
            f.write(f'No duplicate CIDs with different InChIs in AID{AID}\n')
        else:
            f.write('Found duplicate CIDs with different InChIs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameCID_differentInChI))
            f.write("\n")
        
        if sameCID_differentSMILES == []:
            f.write(f'No duplicate CIDs with different SMILES in AID{AID}\n')
        else:
            f.write(f'Found duplicate CIDs with different SMILES in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameCID_differentSMILES))
            f.write("\n")
        f.write("===\n")

## 4.3. Same InChI but with different CIDs or SMILES

In [19]:
#reindex
for AID in AIDs: 
    exec(f"AID{AID}_duplicates_InChI.reset_index(drop=True, inplace=True)")

In [20]:
with open(f'{data_folder}/before_finished/step_4/sameInChI_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameInChI_differentCID = []
        sameInChI_differentSMILES = []
        duplicates_InChI = eval(f'AID{AID}_duplicates_InChI')
        for i in range(len(duplicates_InChI['InChI'])):
            for j in range(i+1, len(duplicates_InChI['InChI'])):
                if duplicates_InChI['InChI'][i] == duplicates_InChI['InChI'][j]:
                    if duplicates_InChI['PUBCHEM_CID'][i] != duplicates_InChI['PUBCHEM_CID'][j]:
                        sameInChI_differentCID.append((duplicates_InChI['PUBCHEM_CID'][i], duplicates_InChI['PUBCHEM_CID'][j]))
                    if duplicates_InChI['PUBCHEM_EXT_DATASOURCE_SMILES'][i] != duplicates_InChI['PUBCHEM_EXT_DATASOURCE_SMILES'][j]:
                        sameInChI_differentSMILES.append((duplicates_InChI['PUBCHEM_CID'][i], duplicates_InChI['PUBCHEM_CID'][j]))
        
        if sameInChI_differentCID == []:
            f.write(f'No duplicate InChIs with different CIDs in AID{AID}\n')
        else:
            f.write('Found duplicate InChIs with different CIDs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameInChI_differentCID))
            f.write("\n")
        
        if sameInChI_differentSMILES == []:
            f.write(f'No duplicate InChIs with different SMILES in AID{AID}\n')
        else:
            f.write(f'Found duplicate InChIs with different SMILES in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameInChI_differentSMILES))
            f.write("\n")
        f.write("===\n")


## 4.5 Same SMILES but with different CIDs or SMILES

In [21]:
#reindex
for AID in AIDs: 
    exec(f"AID{AID}_duplicates_SMILES.reset_index(drop=True, inplace=True)")

In [22]:
with open(f'{data_folder}/before_finished/step_4/sameSMILES_different_others.txt', 'w') as f:
    for AID in AIDs: 
        sameSMILES_differentCID = []
        sameSMILES_differentInChI = []
        duplicates_SMILES = eval(f'AID{AID}_duplicates_SMILES')
        for i in range(len(duplicates_SMILES['PUBCHEM_EXT_DATASOURCE_SMILES'])):
            for j in range(i+1, len(duplicates_SMILES['PUBCHEM_EXT_DATASOURCE_SMILES'])):
                if duplicates_SMILES['PUBCHEM_EXT_DATASOURCE_SMILES'][i] == duplicates_SMILES['PUBCHEM_EXT_DATASOURCE_SMILES'][j]:
                    if duplicates_SMILES['PUBCHEM_CID'][i] != duplicates_SMILES['PUBCHEM_CID'][j]:
                        sameSMILES_differentCID.append((duplicates_SMILES['PUBCHEM_CID'][i], duplicates_SMILES['PUBCHEM_CID'][j]))
                    if duplicates_SMILES['InChI'][i] != duplicates_SMILES['InChI'][j]:
                        sameSMILES_differentInChI.append((duplicates_SMILES['PUBCHEM_CID'][i], duplicates_SMILES['PUBCHEM_CID'][j]))
        
        if sameSMILES_differentCID == []:
            f.write(f'No duplicate SMILES with different CIDs in AID{AID}\n')
        else:
            f.write(f'Found duplicate SMILES with different CIDs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameSMILES_differentCID))
            f.write("\n")
        
        if sameSMILES_differentInChI == []:
            f.write(f'No duplicate SMILES with different InChIs in AID{AID}\n')
        else:
            f.write(f'Found duplicate SMILES with different InChIs in AID{AID}:\n')
            f.write('\n'.join(str(item) for item in sameSMILES_differentInChI))
            f.write("\n")
        f.write("===\n")


## 4.5 Drop duplicates

When dropping duplicates, we will keep the first molecule in a pair or a group of duplicates. For example, here there are 12 duplicates (6 pairs) so we keep 6 of them.

In [23]:
# Keep only the first duplicate in the dataframes:
for AID in AIDs: 
    exec(f"AID{AID}.drop_duplicates(subset=['InChI'], keep='first', inplace=True)")

    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=['InChI'], keep=False)]) == 0:
    print('No more duplicate InChI in AID{AID}')
else:
    print('There are still duplicate InChI in AID{AID}')   
    """
    exec(last_check)

No more duplicate InChI in AID602208
No more duplicate InChI in AID493221
No more duplicate InChI in AID493222
No more duplicate InChI in AID504840
No more duplicate InChI in AID488975
No more duplicate InChI in AID588401


In [24]:
for AID in AIDs: 
    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]) == 0:
    print('No more duplicate SMILES in AID{AID}')
else:
    print('There are still duplicate SMILES in AID{AID}')   
    """
    exec(last_check)

No more duplicate SMILES in AID602208
No more duplicate SMILES in AID493221
No more duplicate SMILES in AID493222
No more duplicate SMILES in AID504840
No more duplicate SMILES in AID488975
No more duplicate SMILES in AID588401


In [25]:
for AID in AIDs: 
    last_check = f"""
if len(AID{AID}[AID{AID}.duplicated(subset=['PUBCHEM_CID'], keep=False)]) == 0:
    print('No more duplicate CID in AID{AID}')
else:
    print('There are still duplicate CID in AID{AID}')   
    """
    exec(last_check)

No more duplicate CID in AID602208
No more duplicate CID in AID493221
No more duplicate CID in AID493222
No more duplicate CID in AID504840
No more duplicate CID in AID488975
No more duplicate CID in AID588401


In [26]:
# Save the dataframes to csv:
for AID in AIDs: 
    exec(f"AID{AID}.to_csv(r'{data_folder}/before_finished/step_4/AID{AID}.csv', index=False)")

# 5. Hierarchical Curation

For the hierarchical curation, there are some rules: 

(1) All assays used should be on the same or close species/cell lines. Optimally, they should also be from the same project/laboratory.

(2) Primary actives (PrA) will have a large false-positive rate. Therefore, they should be tested in follow-up confirmatory screens (optimally dose-reponse). 

(3) Actives could be promiscuous. Therefore, it is optimal to have counter-screens on different targets to test specificity.

(4) For some projects, compounds were tested in multiple rounds. Therefore, assays often have hierarchical relations. From a single primary screen (Pr), active compounds (Pr_A) could be tested in multiple rounds of confirmatory screens (Cf_1, Cf_2, ..., Cf_final) or counter screens (Ct_1, Ct_2, etc.). Actives from confirmatory screens (Cf_A) have a higher possibility of being true active. If an active compound is tested active in counter screens (Ct_A), it is likely to be a promiscuous compound and should not be included. 

(4) It is important to know the relationship between assays. Active sets from downstream screens always have a lower false-positive rate than active sets from upstream screens due to better assay technologies on a smaller set of compounds. Therefore, final hits should be taken from the intersection of the very last confirmatory assays, without tested active in any counter-screens: 
Final hits = [Cf_final1_A ∩ Cf_final2_A ∩ ...] \ [Ct_1_A ∪ Ct_2_A ∪ ...]

However, if the confirmatory assays are unrelated (tested on different set of compounds), then we might have to take the union of their active sets instead of the intersections as in this formula.

(5) The hierarchical relations should be inspected carefully to see if follow-up confirmatory screens include extra compounds (Ex) that were not tested in earlier screens or tested inactive in earlier screens. If exist, these compounds require manual inspection. 

(6) Final inactives should be taken from primary inactives (Pr_I) (not inconclusive, unspecified, or probes), plus extra compounds that were tested inactive in conformatory screens (Ex_I), if justified.
Final inactives = PrI ∪ Ex_I



## 5.1. Classify groups of compounds in each assay by activities

In [27]:
path = f'{data_folder}/before_finished/step_4' 
keynumbers = [488975, 504840, 493221, 588401, 602208, 493222] # specify the keynumbers you want to import

for keynumber in keynumbers:
    filename = os.path.join(path, f'AID{keynumber}.csv')
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=None, header=0)
        exec(f'AID{keynumber} = df')
        exec(f'AID{keynumber}_active = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Active"]')
        exec(f'AID{keynumber}_inactive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inactive"]')
        exec(f'AID{keynumber}_inconclusive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inconclusive"]')
        exec(f'AID{keynumber}_unspecified = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Unspecified"]')
        exec(f'AID{keynumber}_probe = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Probe"]')

In [28]:
#Create a df with first column the variables name, and the second column the number of rows:
df = pd.DataFrame(columns=['AID', 'Tested Compounds', 'Active', 'Inactive', 'Inconclusive', 'Unspecified', 'Probe'])
for keynumber in keynumbers:
    exec(f'df.loc[len(df)] = ["AID{keynumber}", len(AID{keynumber}), len(AID{keynumber}_active), len(AID{keynumber}_inactive), len(AID{keynumber}_inconclusive), len(AID{keynumber}_unspecified), len(AID{keynumber}_probe)]')
df

,AID,Tested Compounds,Active,Inactive,Inconclusive,Unspecified,Probe
0,AID488975,306475,2634,303841,0,0,0
1,AID504840,1544,891,648,5,0,0
2,AID493221,2437,1714,723,0,0,0
3,AID588401,393,274,118,1,0,0
4,AID602208,21,14,7,0,0,0
5,AID493222,2437,94,2343,0,0,0


## 5.2. Check the hierachical relations

In [32]:
def check_is_in(downstream, upstream): 
    downstream_in_upstream = downstream[downstream['PUBCHEM_CID'].isin(upstream['PUBCHEM_CID'])]
    downstream_notin_upstream = downstream[~downstream['PUBCHEM_CID'].isin(upstream['PUBCHEM_CID'])]
    return downstream_in_upstream, downstream_notin_upstream

### Flow: AID488975 (Pr), AID504840 (Cf_a), AID493221 (Cf_b), AID588401 (Cf_c), AID602208 (Cf_d), AID493222 (Ct)

In [36]:
# Check between AID504840 and AID488975
a1, a2 = check_is_in(AID504840, AID488975_inactive)
a3, a4 = check_is_in(a1, AID504840_active)
a5, a6 = check_is_in(AID504840, AID488975)
a7, a8 = check_is_in(a6, AID504840_active)
print(f'Among AID504840, {len(a1)} were tested inactive in AID488975. Among these, {len(a3)} became active')
print(f'Among AID504840, {len(a6)} were not tested in the AID488975. Among these, {len(a7)} became active')

# Check between AID493221 and AID488975
b1, b2 = check_is_in(AID493221, AID488975_inactive)
b3, b4 = check_is_in(a1, AID493221_active)
b5, b6 = check_is_in(AID493221, AID488975)
b7, b8 = check_is_in(a6, AID493221_active)
print(f'Among AID493221, {len(b1)} were tested inactive in AID488975. Among these, {len(b3)} became active')
print(f'Among AID493221, {len(b6)} were not tested in the AID488975. Among these, {len(b7)} became active')

# Check between AID588401 and AID488975
c1, c2 = check_is_in(AID588401, AID488975_inactive)
c3, c4 = check_is_in(a1, AID588401_active)
c5, c6 = check_is_in(AID588401, AID488975)
c7, c8 = check_is_in(a6, AID588401_active)
print(f'Among AID588401, {len(c1)} were tested inactive in AID488975. Among these, {len(c3)} became active')
print(f'Among AID588401, {len(c6)} were not tested in the AID488975. Among these, {len(c7)} became active')

# Check between AID602208 and AID488975
d1, d2 = check_is_in(AID602208, AID488975_inactive)
d3, d4 = check_is_in(a1, AID602208_active)
d5, d6 = check_is_in(AID602208, AID488975)
d7, d8 = check_is_in(a6, AID602208_active)
print(f'Among AID602208, {len(d1)} were tested inactive in AID488975. Among these, {len(d3)} became active')
print(f'Among AID602208, {len(d6)} were not tested in the AID488975. Among these, {len(d7)} became active')

Among AID504840, 17 were tested inactive in AID488975. Among these, 8 became active
Among AID504840, 0 were not tested in the AID488975. Among these, 0 became active
Among AID493221, 38 were tested inactive in AID488975. Among these, 17 became active
Among AID493221, 0 were not tested in the AID488975. Among these, 0 became active
Among AID588401, 2 were tested inactive in AID488975. Among these, 1 became active
Among AID588401, 0 were not tested in the AID488975. Among these, 0 became active
Among AID602208, 0 were tested inactive in AID488975. Among these, 0 became active
Among AID602208, 0 were not tested in the AID488975. Among these, 0 became active


Some of the primary inactives got re-tested in the confirmatory screens AID504840, AID493221, and AID588401. Some among those became active.
The potential final hits should be any compound that were active in all confirmatory screens: AID504840, AID493221, and AID588401 but not active in the counter-screen AID493222 (we ignore AID602208 due to small chemical space). Inactives should be taken from the primary inactives, subtracted by any re-tested compounds that became active. 

## 5.3. Export the data

In [64]:
set_504840 = set(AID504840_active['PUBCHEM_CID'])
set_493221 = set(AID493221_active['PUBCHEM_CID'])
set_588401 = set(AID588401_active['PUBCHEM_CID'])
confirmed_hits_cids = set_504840.intersection(set_493221, set_588401)

confirmed_hits = AID504840_active[AID504840_active['PUBCHEM_CID'].isin(confirmed_hits_cids)]
confirmed_hits = confirmed_hits[~confirmed_hits['PUBCHEM_CID'].isin(AID493222_active['PUBCHEM_CID'])]
confirmed_hits = confirmed_hits.drop_duplicates(subset=['PUBCHEM_CID'], keep='first')
potential_hits = confirmed_hits
print(f'We end up with {len(confirmed_hits)} potential hits.')

We end up with 270 potential hits.


In [65]:
not_inactives_cids = pd.concat([a3['PUBCHEM_CID'], b3['PUBCHEM_CID'], c3['PUBCHEM_CID']]).tolist()
potential_inactives = AID488975_inactive[~AID488975_inactive['PUBCHEM_CID'].isin(not_inactives_cids)]
potential_inactives = potential_inactives.drop_duplicates(subset=['PUBCHEM_CID'], keep='first')
print(f'We end up with {len(potential_inactives)} potential inactives.')

We end up with 303824 potential inactives.


In [66]:
if not os.path.exists(f'{data_folder}/before_finished/step_5'):
    os.makedirs(f'{data_folder}/before_finished/step_5')

In [67]:
#export the potential hits and inactives to csv:
potential_hits.to_csv(f'{data_folder}/before_finished/step_5/potential_hits.csv', index=False)
potential_inactives.to_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', index=False)

# 6. RDkit check

In [68]:
potential_hits = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_hits.csv', sep=',', header=0)
potential_inactives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', sep=',', header=0)

In [69]:
def process_smiles(df):
    """
    This function check if the SMILES strings from a given dataset could be parsed by RDKit or if they returns any problems detected by RDkit
    Input: 
        Pandas dataframe
    Output: 
        mol_list: dictionary with CID as key and RDKit molecule object as value
        problem_list: list of problems detected by RDKit
        cannot_parse: list of CIDs that could not be parsed by RDKit
    """
    mol_list = {}
    problem_list = []
    cannot_parse = []

    for i in df['PUBCHEM_CID']:

        #convert each SMILES to molecule:
        m = Chem.MolFromSmiles(df[df['PUBCHEM_CID'] == i]['PUBCHEM_EXT_DATASOURCE_SMILES'].values[0], sanitize=True)
        mol_list[i] = m

        if m is None:
            cannot_parse.append(i) #save if molecule is non-parsable
            
        elif m is not None:
            problems = Chem.DetectChemistryProblems(m) #identify and capture error messages when creating mol objects.
        if problems != ():
            problem_list.append(problems)
            
    if len(problem_list) > 0: 
        print(problem_list)
    else:
        print("No problems detected")
    return mol_list, problem_list, cannot_parse

In [70]:
mol_hits, problem_list_hits, cannot_parse_hits = process_smiles(potential_hits)
mol_inactives, problem_list_inactives, cannot_parse_inactives = process_smiles(potential_inactives)

No problems detected


[02:26:32] WARNING: not removing hydrogen atom without neighbors
[02:26:35] WARNING: not removing hydrogen atom without neighbors


No problems detected


In [71]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_6'):
    os.makedirs(f'{data_folder}/before_finished/step_6')

with open(f'{data_folder}/before_finished/step_6/problem_list_hits.txt', 'w') as f:
    f.write("Problems:\n")
    for item in problem_list_hits:
        f.write("%s\n" % item)
    f.write("Cannot parse:\n")
    for item in cannot_parse_hits:
        f.write("%s\n" % item)

with open(f'{data_folder}/before_finished/step_6/problem_list_inactives.txt', 'w') as f:
    f.write("Problems:\n")
    for item in problem_list_inactives:
        f.write("%s\n" % item)
    f.write("Cannot parse:\n")
    for item in cannot_parse_inactives:
        f.write("%s\n" % item)

Our dataset returned no problem or non-parsable molecule.

# 7. Inorganics

In [72]:
# Import data
potential_hits = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_hits.csv', sep=',', header=0)
potential_inactives = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives.csv', sep=',', header=0)

In [73]:
organic_hits = []
inorganic_hits = []

for index, row in potential_hits.iterrows():
    cid = row['PUBCHEM_CID']
    smi = row['PUBCHEM_EXT_DATASOURCE_SMILES']
    mol = Chem.MolFromSmiles(smi, sanitize=True)
    if functional_groups.is_inorganic(mol):
        inorganic_hits.append(cid)
    else:
        organic_hits.append(cid)

In [74]:
organic_inactives = []
inorganic_inactives = []

for index, row in potential_inactives.iterrows():
    cid = row['PUBCHEM_CID']
    smi = row['PUBCHEM_EXT_DATASOURCE_SMILES']
    mol = Chem.MolFromSmiles(smi, sanitize=True)
    if functional_groups.is_inorganic(mol):
        inorganic_inactives.append(cid)
    else: 
        organic_inactives.append(cid)

[02:31:16] WARNING: not removing hydrogen atom without neighbors
[02:31:16] WARNING: not removing hydrogen atom without neighbors
[02:31:18] WARNING: not removing hydrogen atom without neighbors
[02:31:18] WARNING: not removing hydrogen atom without neighbors


In [75]:
print(f'In hits, there are {len(organic_hits)} organic molecules and {len(inorganic_hits)} inorganic molecules')
print(f'In inactives, there are {len(organic_inactives)} organic molecules and {len(inorganic_inactives)} inorganic molecules')

In hits, there are 270 organic molecules and 0 inorganic molecules
In inactives, there are 303820 organic molecules and 4 inorganic molecules


In [76]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_7'):
    os.makedirs(f'{data_folder}/before_finished/step_7')

with open(f'{data_folder}/before_finished/step_7/inorganic.txt', 'w') as f:
    f.write("Hits:\n")
    for item in inorganic_hits:
        f.write("%s\n" % item)
    f.write("\n\nInactives:\n")
    for item in inorganic_inactives:
        f.write("%s\n" % item)

In [77]:
# Drop inorganics: 
potential_hits = potential_hits[~potential_hits['PUBCHEM_CID'].isin(inorganic_hits)]
potential_inactives = potential_inactives[~potential_inactives['PUBCHEM_CID'].isin(inorganic_inactives)]


In [78]:
#save: 
potential_hits.to_csv(f'{data_folder}/before_finished/step_7/organic_hits.csv', index=False)
potential_inactives.to_csv(f'{data_folder}/before_finished/step_7/organic_inactives.csv', index=False)

# 8. Mixture

## 8.1. Quick check

In [79]:
#import: 
organic_hits = pd.read_csv(f'{data_folder}/before_finished/step_7/organic_hits.csv', sep=',', header=0)
organic_inactives = pd.read_csv(f'{data_folder}/before_finished/step_7/organic_inactives.csv', sep=',', header=0)

In [80]:
def quick_check_mixtures(name, smiles_list):
    count=0
    for smiles in smiles_list:
        if '.' in smiles:
            count+=1
    print(f"Total number of mixtures in {name} is {count}")

quick_check_mixtures('hits', organic_hits['PUBCHEM_EXT_DATASOURCE_SMILES'])
quick_check_mixtures('inactives', organic_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'])

Total number of mixtures in hits is 31
Total number of mixtures in inactives is 12093


## 8.2. Handling mixture

In [81]:
def process_smiles_dataframe(df):
    """
    From a given dataframe, detect and handle mixtures based on SMILES representation.
    Input: 
        Pandas dataframe
    Output: 
        (dictionary: CID -> SMILES)
            processed: non-mixture forms of every SMILES in the given dataset
            removed: mixture components or mixtures removed from the original mixture SMILES
            small_organic: small organic molecules that are removed from a size-imbalanced mixtures of organic molecules
            small_inorganic: small inorganic molecules that are removed from a mixture of both organic and inorganic molecules
            big_organic_not_lipinski: large organic molecules that are not kept due to not passing the lipinski criteria
            cleaned_from_mixtures: non-mixture forms after handling of the orignal mixtures
    """
    processed = {}
    removed = {}
    small_inorganic = {}
    small_organic = {}
    big_organic_not_lipinski = {}
    cleaned_from_mixtures = {}
    
    for index, row in df.iterrows():
        cid = row['PUBCHEM_CID']
        smiles = row['PUBCHEM_EXT_DATASOURCE_SMILES']
        
        if '.' in smiles: # Check for mixtures
            molecules = smiles.split('.')
            mols = [Chem.MolFromSmiles(mol) for mol in molecules]
            num_atoms = [mol.GetNumAtoms() for mol in mols if mol is not None]
            
            if all(x == num_atoms[0] for x in num_atoms): # Check if all molecules have the same number of atoms. If yes, keep one of them
                processed[cid] = molecules[0]
                cleaned_from_mixtures[cid] = molecules[0]
                removed[cid] = '.'.join(molecules[1:])
            else:
                if max(num_atoms) - min(num_atoms) <= 5:
                    removed[cid] = smiles # Remove the mixture if the difference in number of atoms is less than 5
                    print(f"Cannot decide between {molecules} for CID {cid}")
                else:
                    max_index = num_atoms.index(max(num_atoms))
                    min_index = num_atoms.index(min(num_atoms))
                    if functional_groups.is_inorganic(mols[min_index]) == True:
                        processed[cid] = molecules[max_index]
                        cleaned_from_mixtures[cid] = molecules[max_index]
                        removed[cid] = molecules[min_index] # Keep the organic molecule and remove the inorganic one
                        small_inorganic[cid] = molecules[min_index]
                    else:
                        big_molecule = mols[max_index]
                        
                        # Calculate properties for Lipinski's rule of five
                        mw = Descriptors.MolWt(big_molecule)
                        hbd = rdMolDescriptors.CalcNumHBD(big_molecule)
                        hba = rdMolDescriptors.CalcNumHBA(big_molecule)
                        logp = Crippen.MolLogP(big_molecule)

                        # Check Lipinski's criteria
                        if mw <= 500 and hbd <= 5 and hba <= 10 and logp <= 5:
                            processed[cid] = molecules[max_index] # Keep the big organic molecule if it passes Lipinski's rule of five
                            cleaned_from_mixtures[cid] = molecules[max_index]
                            removed[cid] = '.'.join([molecules[i] for i in range(len(molecules)) if i != max_index])
                            small_organic[cid] = molecules[min_index]
                        else:
                            removed[cid] = smiles # Remove the mixture if the big organic molecule does not pass Lipinski's rule of five
                            big_organic_not_lipinski[cid] = molecules[max_index]
                            print(f"Big organic molecule for CID {cid} does not pass Lipinski's rule of five")

        else:
            processed[cid] = smiles
            
    return processed, removed, small_organic, small_inorganic, big_organic_not_lipinski, cleaned_from_mixtures


In [82]:
processed_hits, removed_hits, small_organic_hits, small_inorganic_hits, not_lipinski_hits, cleaned_hits = process_smiles_dataframe(organic_hits)
processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives, not_lipinski_inactives, cleaned_inactives = process_smiles_dataframe(organic_inactives)

Cannot decide between ['CN1CCCN=C1/C=C/C2=CC=CS2', 'C(C(C(=O)O)O)(C(=O)O)O'] for CID 6419953
Cannot decide between ['C[N+](C)(CCOC1=CC=CC=C1)CC2=CC=CC=C2', 'C1=CC=C2C(=C1)C=C(C(=C2Br)[O-])C(=O)O'] for CID 54732065
Big organic molecule for CID 14219807 does not pass Lipinski's rule of five
Big organic molecule for CID 12005527 does not pass Lipinski's rule of five
Big organic molecule for CID 12005684 does not pass Lipinski's rule of five
Big organic molecule for CID 12005677 does not pass Lipinski's rule of five
Cannot decide between ['CC1=CC=C(C=C1)S(=O)(=O)[O-]', 'CN1C=C[N+](=C1COC(=O)N(C)C)C'] for CID 15945791
Big organic molecule for CID 15945428 does not pass Lipinski's rule of five
Big organic molecule for CID 15944995 does not pass Lipinski's rule of five
Big organic molecule for CID 15945399 does not pass Lipinski's rule of five
Cannot decide between ['CC1=CC=C(C=C1)S(=O)(=O)[O-]', 'CN1C=C[N+](=C1CCOC(=O)N(C)C)C'] for CID 15945641
Cannot decide between ['CCCCC1=NC2=CC=CC=C2C(=C

[02:34:25] WARNING: not removing hydrogen atom without neighbors
[02:34:25] WARNING: not removing hydrogen atom without neighbors
[02:34:25] WARNING: not removing hydrogen atom without neighbors
[02:34:25] WARNING: not removing hydrogen atom without neighbors


Big organic molecule for CID 5718631 does not pass Lipinski's rule of five
Big organic molecule for CID 135433258 does not pass Lipinski's rule of five
Cannot decide between ['CCCC1=NCCN2C1=CC=C2', 'C(=C/C(=O)O)\\C(=O)O'] for CID 5717182
Big organic molecule for CID 5729990 does not pass Lipinski's rule of five
Cannot decide between ['CC1=NC2=C(O1)C3=CC=CC=C3C=C2', 'C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]'] for CID 3241713
Big organic molecule for CID 135501964 does not pass Lipinski's rule of five
Cannot decide between ['CN(C)C1=NC2=C(CCC2)C(=C1)N', 'C1=C(C=NC=C1O)C(=O)O'] for CID 652177
Big organic molecule for CID 135512395 does not pass Lipinski's rule of five
Big organic molecule for CID 15944957 does not pass Lipinski's rule of five
Big organic molecule for CID 9549148 does not pass Lipinski's rule of five
Big organic molecule for CID 9549634 does not pass Lipinski's rule of five
Big organic molecule for CID 9549642 does not pass Lipinski's rule of five
Big organic

In [83]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_8'):
    os.makedirs(f'{data_folder}/before_finished/step_8')

In [84]:
#Generate df with the smiles column in the cleaned_hits or cleaned_inactives dictionary:
cleaned_hits_df = pd.DataFrame(list(cleaned_hits.values()), columns=['PUBCHEM_EXT_DATASOURCE_SMILES'])
cleaned_inactives_df = pd.DataFrame(list(cleaned_inactives.values()), columns=['PUBCHEM_EXT_DATASOURCE_SMILES'])

In [85]:
#Export the cleaned hits and inactives to csv:
cleaned_hits_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_hits.csv', index=False, header=False)
cleaned_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_inactives.csv', index=False, header=False)

In [86]:
def process_mixture_df(name, df, processed, removed, small_organic, small_inorganic):
    """
    Update a given dataframe with information on mixture handling
    """
    indices_to_drop = []  # List to keep track of row indices that should be dropped
    
    for index, row in df.iterrows():
        cid = row['PUBCHEM_CID']
        if cid in processed:
            df.loc[index, 'PUBCHEM_EXT_DATASOURCE_SMILES'] = processed[cid]
            if cid in removed:
                df.loc[index, 'Mol removed from mixture'] = removed[cid]
            if cid in small_organic:
                df.loc[index, 'Small organic molecule'] = small_organic[cid]
            if cid in small_inorganic:
                df.loc[index, 'Small inorganic molecule'] = small_inorganic[cid]
        else:
            indices_to_drop.append(index)
            print(f"{cid} has been removed from {name} because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.")
    
    # Drop rows outside the loop and reset index if needed
    new_df = df.drop(indices_to_drop).reset_index(drop=True)
    
    return new_df

processed_hits_df = process_mixture_df('hits_M1_antagonist', organic_hits, processed_hits, removed_hits, small_organic_hits, small_inorganic_hits)
processed_inactives_df = process_mixture_df('inactives_M1_antagonist', organic_inactives, processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives)

6419953 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
54732065 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
14219807 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
12005527 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
12005684 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
12005677 has been removed from inactives_M1_antagonist because it is a mixture with less than 5 atoms

In [87]:
with open(f'{data_folder}/before_finished/step_8/mixture.txt', 'w') as f:
    f.write(f"""
Hits before processing: {len(organic_hits)}
Hits before processing: {len(organic_hits)}
Hits after processing: {len(processed_hits_df)}
Mixtures detected: {len(removed_hits)}
Mixtures with small inorganic molecules: {len(small_inorganic_hits)}
Mixtures with big organic molecules passing Lipinski: {len(small_organic_hits)}
Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_hits)}

Inactives before processing: {len(organic_inactives)}
Inactives after processing: {len(processed_inactives_df)}
Mixtures detected: {len(removed_inactives)}
Mixtures with small inorganic molecules: {len(small_inorganic_inactives)}
Mixtures with big organic molecules passing Lipinski: {len(small_organic_inactives)}
Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_inactives)}
""")

# Save the processed dataframes to csv
processed_hits_df.to_csv(f'{data_folder}/before_finished/step_8/post8_hits.csv', index=False)
processed_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', index=False)

print('Dataframes saved successfully')

Dataframes saved successfully


# 9. Neutralize & 10. Aromatize molecules

In [88]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_9_10'):
    os.makedirs(f'{data_folder}/before_finished/step_9_10')

In [89]:
#import:
pre9_hits = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_hits.csv', sep=',', header=0)
pre9_inactives = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', sep=',', header=0)

In [90]:
def neutralize_atoms(mol):
    """
    Code adapted from https://www.rdkit.org/docs/Cookbook.html. 
    Source: https://baoilleach.blogspot.com/2019/12/no-charge-simple-approach-to.html
    (Noel O’Boyle, 2019)

    This function return a neutralized molecules for a given input Mol object. 
    Additional handling was added for molecules with tetracoordinated boron. 
    """
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            
            #Skip adjustment for tetracoordinated boron
            if atom.GetAtomicNum() == 5 and atom.GetDegree() == 4: #ADD COMMENT
                continue  # Just bypass the problematic atom

            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

In [91]:
def aromatize_smile(mol):
    """
    This function dekekulize an input Mol object and return the aromatic form of isomeric SMILES. 
    """
    Chem.Kekulize(mol)
    Chem.SanitizeMol(mol, Chem.SanitizeFlags.SANITIZE_ALL)
    aromatic_smiles = Chem.MolToSmiles(mol, isomericSmiles = True)
    return aromatic_smiles

In [92]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES']: 
    mol = Chem.MolFromSmiles(smi)
    mol_neu = neutralize_atoms(mol)
    smi_arom = aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)
    
#update the smiles in this df
pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'] = updated_smi

In [93]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES']: 
    mol = Chem.MolFromSmiles(smi)
    mol_neu = neutralize_atoms(mol)
    smi_arom = aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)

#update the smiles in this df
pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'] = updated_smi

In [94]:
#Save
pre9_hits.to_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# Post 9+10: Update InChI


Is it important to now update InChI in our datasets, for 2 reasons:

(1) Some mixture compounds have been modified (removal of small inorganic or organic molecules) in SMILES representation but not InChIs.

(2) The SMILES representations have been neutralized and aromatized, but not InChIs.

In [95]:
#export the smiles columns to txt
pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_hits.txt', index=False, header=False)
pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_inactives.txt', index=False, header=False)

In [96]:
"""
Submit the smiles files to PubChem Identifier Exchange Service: 
    Input IDs: "SMILES"
    Operator type: "same CID" 
    Output IDs: "InChI"
    Output method: "Two column file showing each input output-correspondence"
    Compression: "No compression"
InChI list should be saved into "step_9_10" folder, named as "inchi_hits.txt" and "inchi_inactives" 
"""
#Import the converted InChIs
cleaned_inchi_hits = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_hits.txt', sep='\t', header=None)
cleaned_inchi_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_inactives.txt', sep='\t', header=None)

#a dictionary of smiles and corresponding inchi in cleaned_inchi_hits
hits_smi_inchi_dict = dict(zip(cleaned_inchi_hits[0], cleaned_inchi_hits[1]))
inactives_smi_inchi_dict = dict(zip(cleaned_inchi_inactives[0], cleaned_inchi_inactives[1]))
                             
#update the pre9_hits by matching the smiles with keys and replace inchi with values:
pre9_hits['InChI'] = pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'].map(hits_smi_inchi_dict) 
pre9_inactives['InChI'] = pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'].map(inactives_smi_inchi_dict)


In [97]:
#export: 
pre9_hits.to_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# 11. PAIN filters

## 11.1. Frequency of hits (FoH)

Frequency of Hits is a complex concept that requires a merticulous approach. In general, the rule is if a compound was tested active in multiple assays, it is likely to be a promiscuous compound. 
1. For each compounds, retrieve the information on its tested assays
2. For each of the assay tested, retrieve the sequence of the protein target. 
3. Given all sequence of the protein tested, do a multiple sequence alignment to find the percentage Percent Identity (similarty) between these proteins. If an assay has high percentage to other targets, then these assays contribute less to promiscuousity of the compound. 
4. Use the percentage identity as a weight: 
w = 1 - %SI/100
Calculate the frequency of hits for each compound:
FoH = wACC/TAC
wACC is the weighed total number of assay tested where the compounds were identified acitives. TAC is the total number of assays tested. 

In [98]:
pre11_hits = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', sep=',', header=0)
pre11_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', sep=',', header=0)

In [99]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_1'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_1')

### 11.1.1. PubChem testing information for each compound

This part illustrates how to retrieve the information of how each compound was tested from the PubChem database. Bulk data retrieval from the ftp server is used to get the information of every bioassay in PubChem:

In [77]:
url = 'https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Extras/bioassays.tsv.gz' #this FTP file records the summary data of all available AIDs in PubChem

local_save_dir = 'H:/coding/HiChem/curation/pubchem_sum'
local_save_path = os.path.join(local_save_dir, 'bioassays.tsv.gz')

if not os.path.exists(local_save_dir):
    os.makedirs(local_save_dir)
r = requests.get(url, stream=True)

with open(local_save_path, 'wb') as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
print('Downloaded to %s' % local_save_path)

Downloaded to H:/coding/HiChem/curation/pubchem_sum\bioassays.tsv.gz


In [100]:
path = 'pubchem_sum/bioassays.tsv.gz'

# Read the TSV file
all_bioassay = pd.read_csv(path, delimiter='\t')

In [101]:
all_bioassay.head()

,AID,BioAssay Name,Deposit Date,Modify Date,Source Name,Source ID,Substance Type,Outcome Type,Project Category,BioAssay Group,BioAssay Types,Protein Accessions,UniProts IDs,Gene IDs,Target TaxIDs,Taxonomy IDs,Number of Tested SIDs,Number of Active SIDs,Number of Tested CIDs,Number of Active CIDs
0,1,NCI human tumor cell line growth inhibition as...,20040815,20240410,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,55228,3318,53214,3094
1,3,NCI human tumor cell line growth inhibition as...,20040815,20240410,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,51435,2615,49564,2467
2,5,NCI human tumor cell line growth inhibition as...,20040815,20240410,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,54079,2503,52046,2317
3,7,NCI human tumor cell line growth inhibition as...,20040815,20240410,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,54062,4335,52033,4098
4,9,NCI human tumor cell line growth inhibition as...,20040815,20240410,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53977,3159,52001,2981


### 11.1.2 Retrieving protein sequences for assays tested:

Then, the testing information for each compound is retrieved from the PugREST API

In [102]:
# Cache to store the number of compounds tested per AID to avoid redundant call. 
num_compounds_tested_cache = {}

def get_num_compounds_tested(aid, all_bioassay=all_bioassay):
    """
    This function retrieves the information of how many compounds were tested in a given assay (by AID).
    """
    if aid in num_compounds_tested_cache:
        return num_compounds_tested_cache[aid]
    else: 
        #return the 'Number of Tested CIDs' column value at the row where the 'AID' column is equal to aid in the all_bioassay dataframe
        num_compounds_tested = all_bioassay[all_bioassay['AID'] == aid]['Number of Tested CIDs'].values[0]
    return num_compounds_tested

def get_assay_data(cid):
    """
    Return a dictionary of all targets that a given compound (by CID) was tested on in PubChem 
    and the activity values of the compound. 
    """
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/assaysummary/JSON" #PUG-REST compound summary by CID
    response = requests.get(url)
    data = response.json()

    target_activity = {}

    if 'Table' in data and 'Row' in data['Table']:
        for row in data['Table']['Row']:
            cells = row['Cell']
            aid = int(cells[0])  # Extracting the AID from the first cell

            # Proceed only if the assay is a screening assay
            if cells[10] == 'Screening':

                # Proceed only if more than 10,000 compounds were tested
                num_compounds_tested = get_num_compounds_tested(aid)
                if num_compounds_tested > 10000:
                    target_gi = cells[5] # Retrieve the protein target's GI
                    activity_outcome = cells[4].lower()

                    if target_gi not in target_activity:
                        target_activity[target_gi] = activity_outcome == 'active'
                    elif activity_outcome == 'active':
                        target_activity[target_gi] = True # If a compound was tested multiple times on the same protein, priotize "active" outcome.
            
            else:
                continue

    return (cid, target_activity)

In [103]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

cids_list = pre11_hits['PUBCHEM_CID'].tolist()

def execute_with_multiprocessing(cids_list):
    """
    For a given list of CIDs, return a dictionary of dictionaries 
    of protein targets these compounds were tested on and the activity outcomes
    Input: 
        [list of CIDs]
    Output: 
        Dictionary of testing information for all CIDs, such as:
        {CID1:{target1:activity1, target3:activity3, ...},{CID2:{target2:activity2, target4:activity4, ...}, ...}}
    """
    results_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Prepare futures for all CIDs
        futures = [executor.submit(get_assay_data, cid) for cid in cids_list]
        
        # Process futures as they complete
        for future in tqdm(as_completed(futures), total=len(cids_list), desc="Processing CIDs"):
            try:
                cid, target_activity = future.result()
                results_dict[cid] = target_activity
            except Exception as e:
                print(f"Error processing CID: {e}")
    return results_dict

results_dict = execute_with_multiprocessing(cids_list)

Processing CIDs:   0%|          | 0/270 [00:00<?, ?it/s]

Processing CIDs: 100%|██████████| 270/270 [02:47<00:00,  1.61it/s]


In [104]:
#export results_dict
with open(f'{data_folder}/before_finished/step_11/11_1/results_dict.json', 'w') as f:
    json.dump(results_dict, f)

In [105]:
#import results_dict:
with open(f'{data_folder}/before_finished/step_11/11_1/results_dict.json', 'r') as f:
    results_dict = json.load(f)

In [106]:
#get the list of all keys of the values in the dictionary:
protein_ids = []
for value in results_dict.values():
    protein_ids.extend(value.keys())

#clean the list
protein_ids = list(set(protein_ids))
protein_ids = [id for id in protein_ids if id != '']

print(protein_ids)
print(f'Require multiple sequencing alignment for {len(protein_ids)} proteins.')

['1654220559', '119607128', '89191863', '38027923', '1572493', '21327705', '4502495', '216409728', '4506113', '25148072', '54112388', '62362414', '34577122', '160707929', '486173', '5454102', '55958172', '15609874', '62740231', '119607129', '13124881', '112822', '116077694', '78070770', '493539358', '4503219', '124809506', '1166512', '5453722', '10567816', '301171662', '487738', '14790119', '83627717', '4507615', '13236497', '30219', '23943882', '11093520', '9937384', '17391426', '56786138', '7669492', '21955158', '730163', '14149746', '187960042', '90652859', '49168602', '124486680', '194068499', '4505447', '68565218', '6679827', '115347926', '55662034', '32307126', '111305821', '48146199', '4757950', '32307152', '56417702', '9629363', '4503779', '130375', '90111653', '536029', '6912644', '4502003', '4503155', '149631', '5729858', '4506243', '37589898', '67463988', '7108463', '62201602', '120538355', '735367775', '510901', '4506055', '167013344', '120997', '6325022', '5016090', '14790

Now we retrieve all the FASTA sequences of proteins tested for all of our compounds with Biopython API to Entrez of NCBI. The FASTA sequence is saved as "sequences.fasta"

In [107]:
# Always tell NCBI who you are
Entrez.email = "hdong26@amherst.edu"

# The filename where you want to save the sequences
output_filename = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'

# Open a file to write the sequences
with open(output_filename, "w") as output_file:
    for id in protein_ids:
        try:
            # Fetch the sequence from NCBI
            handle = Entrez.efetch(db="protein", id=id, rettype="fasta", retmode="text")
            sequence_data = handle.read()
            handle.close()
            
            # Write the sequence data to the file
            output_file.write(sequence_data)
        except Exception as e:
            print(f"An error occurred while fetching {id}: {e}")

From the FASTA sequence, we also need to retrieve the list of protein names, since these are different from the protein GIs

In [108]:
def extract_protein_names(file_path):
    protein_names = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('>'):
                # Split the line at spaces and take the first item
                parts = line.split(' ')
                protein_name = parts[0]
                # Remove the leading '>' character
                protein_name = protein_name[1:]
                protein_names.append(protein_name)
    return protein_names

In [109]:
file_path = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'
protein_names = extract_protein_names(file_path)

In [110]:
# Create a dictionary to map protein IDs to protein names by index 
protein_id_to_name = {protein_ids[i]: protein_names[i] for i in range(len(protein_ids))}

### 11.1.3 Percent Sequence Identity by Multiple Sequence Alignment

The sequences.fasta file is submitted to https://www.ebi.ac.uk/jdispatcher/msa/clustalo for multiple sequencing alignment. The resulted table of percent sequence identity matrix is saved and imported for the calculation of FoH

In [111]:
"""
Submit sequences.fasta to https://www.ebi.ac.uk/jdispatcher/msa/clustalo
    Input sequence type: Protein
    Output format: ClustalW with character counts
Download the resulted Percent Identity Matrix file file and save as "percent_identity_matrix.txt"
"""

#import the identity matrix:
protein_si = pd.read_csv(
    f'{data_folder}/before_finished/step_11/11_1/percent_identity_matrix.txt',
    delimiter='\s+',
    header=None,
    skiprows=6 
)

In [112]:
#remove the first column:
protein_si = protein_si.drop(protein_si.columns[0], axis=1)

In [113]:
name = protein_si[1].tolist()
name = ['protein name'] + name
protein_si.columns = name

In [114]:
protein_si

,protein name,NP_987096.1,NP_004447.2,ABC40742.1,NP_821133.1,AAH93020.1,AAH19268.2,NP_269944.1,AAH03636.1,NP_848757.5,...,sp|Q05DJ8|Q05DJ8_HUMAN,CCE37082.1,CAA96025.1,NP_112168.1,CAC29064.1,sp|P31749.2|AKT1_HUMAN,CAD53427.1,sp|P53779.2|MK10_HUMAN,NP_002084.2,NP_063937.2
0,NP_987096.1,100.00,13.76,15.45,14.18,15.74,15.38,13.42,12.90,10.29,...,9.76,8.57,11.36,7.58,7.58,15.09,17.02,9.76,10.00,9.38
1,NP_004447.2,13.76,100.00,16.75,10.74,8.24,15.06,14.88,10.34,10.79,...,9.72,9.03,8.61,10.97,10.97,8.97,6.00,6.08,10.00,9.82
2,ABC40742.1,15.45,16.75,100.00,11.82,16.67,11.11,11.61,12.12,12.87,...,4.55,9.38,11.11,11.69,11.69,6.56,11.63,8.43,9.46,12.16
3,NP_821133.1,14.18,10.74,11.82,100.00,13.70,13.04,13.25,12.68,13.56,...,6.12,4.55,9.09,4.65,4.65,4.29,3.45,3.95,3.90,5.19
4,AAH93020.1,15.74,8.24,16.67,13.70,100.00,18.60,14.47,10.92,14.39,...,6.67,11.54,8.33,9.76,9.76,3.45,23.33,7.84,13.95,19.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,sp|P31749.2|AKT1_HUMAN,15.09,8.97,6.56,4.29,3.45,0.00,11.76,6.56,9.58,...,16.60,16.34,14.75,29.95,29.95,100.00,8.56,21.39,23.35,22.01
421,CAD53427.1,17.02,6.00,11.63,3.45,23.33,8.00,0.00,9.38,7.84,...,18.45,18.14,16.67,9.63,9.63,8.56,100.00,13.98,16.67,28.21
422,sp|P53779.2|MK10_HUMAN,9.76,6.08,8.43,3.95,7.84,5.77,7.69,11.90,5.52,...,15.27,16.80,13.55,25.70,25.70,21.39,13.98,100.00,25.25,26.52
423,NP_002084.2,10.00,10.00,9.46,3.90,13.95,14.29,6.90,7.14,12.96,...,16.48,18.00,12.30,22.59,22.59,23.35,16.67,25.25,100.00,76.85


### 11.1.4 Calculation of FoHs:

Until now, we have a dictionary of (cid: assays tested); (assay_tested:protein name), and percentage identity matrix with first columns as protein names. 
For each compound, we retrieve the list of all protein names tested on that compounds by matching between the two first dictionary. From this list, we retrieve the corresponding matrix of percentages identitiy of these proteins corresponding to these compounds and calculate the FoH

In [115]:
protein_si_dict = {}
for name in protein_si['protein name']: 
    for other_name in protein_si['protein name']: 
        if other_name != name: 
            protein_si_dict[(name, other_name)] = protein_si.loc[protein_si['protein name'] == name, other_name].values[0]

In [116]:
import tqdm

foh_dict = {}

for cid, targets in tqdm.tqdm(results_dict.items()):
    active_weight_list = []
    total_weight_list = []

    for target_id, result in targets.items():
        if target_id == '':
            continue

        protein_name = protein_id_to_name[target_id]
        max_weight = 0

        for other_id, other_result in targets.items():
            if other_id != target_id and other_id != '':
                other_protein_name = protein_id_to_name[other_id]
                value = protein_si_dict[(protein_name, other_protein_name)]
                max_weight = max(max_weight, value)

        target_weight = 1 - max_weight / 100

        if result:
            active_weight_list.append(target_weight)
        total_weight_list.append(target_weight)

    if total_weight_list:
        foh_score = sum(active_weight_list) / sum(total_weight_list)
        foh_dict[cid] = foh_score
    else: 
        foh_dict[cid] = 0


  0%|          | 0/270 [00:00<?, ?it/s]

100%|██████████| 270/270 [00:15<00:00, 17.51it/s]


In [117]:
#export foh_dict
with open(f'{data_folder}/before_finished/step_11/11_1/foh_dict.json', 'w') as f:
    json.dump(foh_dict, f)

For compounds with FoH larger than 0.26, we remove them

In [118]:
to_drop = []
for cid, foh_score in foh_dict.items():
    if foh_score > 0.26: 
        to_drop.append(cid)

post_FoH_hits = pre11_hits[~pre11_hits['PUBCHEM_CID'].isin(to_drop)]
print(f'Dropped {(len(to_drop))} compounds with FoH larger than 0.26')

Dropped 0 compounds with FoH larger than 0.26


In [119]:
# Save post_FoH_hits: 
post_FoH_hits.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_hits.csv', index=False)

In [120]:
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_inactives.csv', index=False)

## 11.2 Autofluoresence & Luceferase inhibition

When finding false positive due to autofluorescence and luceferase inhibition, it is important to check if the particular assays use one of these technologies. Here, all three assays (AID626, AID1488, and AID1741) use fluorescence technologies, so it is optimal to remove compounds that are active in AIDs: 587, 588, 590, 591, 592, 593, 594

In [121]:
autofluorescence_aids = ['587', '588', '590', '591', '592', '593', '594']
autofluorescence_cids = []
col_list = ['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME']

count = 0
for AID in autofluorescence_aids:
    url = f'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={AID}'
    autofluorescence_df = pd.read_csv(url, usecols=col_list)
    #delete rows with nan values
    autofluorescence_df = autofluorescence_df.dropna(subset=['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME'])

    #convert cids to int: 
    autofluorescence_df['PUBCHEM_CID'] = autofluorescence_df['PUBCHEM_CID'].astype(int)

    #keep only rows said "Active"
    autofluorescence_df = autofluorescence_df[autofluorescence_df['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
    autofluorescence_cids.extend(autofluorescence_df['PUBCHEM_CID'].tolist())


In [122]:
#drop duplicates in the list:
autofluorescence_cids = [item for item in set(autofluorescence_cids)]

In [123]:
to_drop_hits = []
for cid in post_FoH_hits: 
    if cid in autofluorescence_cids:
        to_drop.append(cid)
post_autofluorescence = post_FoH_hits[~post_FoH_hits['PUBCHEM_CID'].isin(to_drop_hits)]
print(f'Dropped {(len(to_drop_hits))} autofluorescence compounds')

Dropped 0 autofluorescence compounds


In [124]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_2'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_2')

In [125]:
#save: 
post_autofluorescence.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_hits.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_inactives.csv', index=False)

## 11.3 RDKit PAIN filter

In [126]:
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
catalog = FilterCatalog(params)

In [127]:
def detect_pains(df):
    pains = []
    count_pains = 0
    count_not_pains = 0
    smiles_column = 'PUBCHEM_EXT_DATASOURCE_SMILES'
    cids_column = 'PUBCHEM_CID'
    for i in df.index:
        smile = str(df.loc[i, smiles_column])
        cid = df.loc[i, cids_column]
        mol = Chem.MolFromSmiles(smile)
        if mol is not None:
            if catalog.HasMatch(mol):
                pains.append(cid)
                count_pains += 1
                print(f'{count_pains} pains detected')
            else: 
                count_not_pains += 1
    return pains

pains = detect_pains(post_autofluorescence)

1 pains detected
2 pains detected
3 pains detected
4 pains detected
5 pains detected
6 pains detected
7 pains detected
8 pains detected
9 pains detected
10 pains detected
11 pains detected
12 pains detected
13 pains detected
14 pains detected
15 pains detected
16 pains detected
17 pains detected
18 pains detected
19 pains detected


In [128]:
post_pains_hits = post_autofluorescence[~post_autofluorescence['PUBCHEM_CID'].isin(pains)]

In [129]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_3'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_3')

post_pains_hits.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_hits.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', index=False)

# 12. Drug-likeness filter: 

In [130]:
pre12_hits = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_hits.csv', sep=',', header=0)
pre12_inactives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', sep=',', header=0)

In [131]:
from tqdm import tqdm

def drug_likeness_filter(smiles):
    """
    This functions check if a given smiles satisfies the common standard conditions for drug-likeness. 
    Input:
        SMILES (str)
    Output: 
        Result (bool)
    """

    # Convert SMILES string to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return False  # Return False if the molecule cannot be parsed
    
    # Check molecular weight
    mw = Chem.rdMolDescriptors.CalcExactMolWt(mol)
    if not (150 < mw < 800):
        return False
    
    # Check AlogP
    logp = Chem.Crippen.MolLogP(mol)
    if not (-0.3 < logp < 5):
        return False
    
    # Check number of rotatable bonds
    rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    if rotatable_bonds >= 15:
        return False
    
    # Check H-bond acceptor count and H-bond donor count
    hba = Lipinski.NumHAcceptors(mol)
    hbd = Lipinski.NumHDonors(mol)
    if hba >= 15 or hbd >= 15:
        return False
    
    # Check total formal charge
    total_charge = sum(atom.GetFormalCharge() for atom in mol.GetAtoms())
    if not (-2 < total_charge < 2):
        return False
    
    # If all filters passed, return True
    return True

def drug_likeness_filter_multiprocessing(df):
    """
    This function update a given dataframe by dropping molecules that don't pass the drug-likeness filter.
    """
    to_drop = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Create future tasks for each SMILES string in the dataframe
        futures = {executor.submit(drug_likeness_filter, row['PUBCHEM_EXT_DATASOURCE_SMILES']): row['PUBCHEM_CID'] for index, row in df.iterrows()}
        
        # Use tqdm to display progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing SMILES"):
            cid = futures[future]
            if not future.result():
                to_drop.append(cid)
    return to_drop

In [132]:
not_drug_hits = drug_likeness_filter_multiprocessing(pre12_hits)

Processing SMILES: 100%|██████████| 251/251 [00:00<00:00, 231734.60it/s]


In [133]:
not_drug_inactives = drug_likeness_filter_multiprocessing(pre12_inactives)

Processing SMILES: 100%|██████████| 303615/303615 [00:03<00:00, 79433.05it/s] 


In [134]:
post12_hits = pre12_hits[~pre12_hits['PUBCHEM_CID'].isin(not_drug_hits)]
post12_inactives = pre12_inactives[~pre12_inactives['PUBCHEM_CID'].isin(not_drug_inactives)]

print(f'Dropped {(len(not_drug_hits))} hit compounds that do not pass the drug likeness filter')
print(f'Dropped {(len(not_drug_inactives))} inactive compounds that do not pass the drug likeness filter')


Dropped 15 hit compounds that do not pass the drug likeness filter
Dropped 15111 inactive compounds that do not pass the drug likeness filter


In [135]:
if not os.path.exists(f'{data_folder}/before_finished/step_12'):
    os.makedirs(f'{data_folder}/before_finished/step_12')

#Export not_drug_hits and inactives:
with open(f'{data_folder}/before_finished/step_12/not_drug_hits.json', 'w') as f:
    json.dump(not_drug_hits, f)
with open(f'{data_folder}/before_finished/step_12/not_drug_inactives.json', 'w') as f:
    json.dump(not_drug_inactives, f)

In [136]:
# save: 
post12_hits.to_csv(f'{data_folder}/before_finished/step_12/post12_hits.csv', index=False)
post12_inactives.to_csv(f'{data_folder}/before_finished/step_12/post12_inactives.csv', index=False)

# 13. ChemBL Curation Pipeline

In [137]:
pre13_hits = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_hits.csv', sep=',', header=0)
pre13_inactives = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_inactives.csv', sep=',', header=0)

In [138]:
def checker_score(smiles, cid):
    result = checker.check_molblock(Chem.MolToMolBlock(Chem.MolFromSmiles(smiles)))
    if result == ():
        penalty_score = 0
    else:
        penalty_score = result[0][0]
    return cid, penalty_score

def checker_multiprocessing(df):
    chembl_score_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Map futures to CIDs directly for easier reference
        futures = {executor.submit(checker_score, row['PUBCHEM_EXT_DATASOURCE_SMILES'], row['PUBCHEM_CID']): row['PUBCHEM_CID'] for _, row in df.iterrows()}
        # Properly use tqdm to create a progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing SMILES"):
            cid, penalty_score = future.result()
            chembl_score_dict[cid] = penalty_score
    return chembl_score_dict

In [139]:
score_hits = checker_multiprocessing(pre13_hits)
score_inactives = checker_multiprocessing(pre13_inactives)

Processing SMILES: 100%|██████████| 236/236 [00:00<00:00, 235008.49it/s]
[04:29:29] Conflicting single bond directions around double bond at index 5.
[04:29:29]   BondStereo set to STEREONONE and single bond directions set to NONE.
[04:29:30] Conflicting single bond directions around double bond at index 4.
[04:29:30]   BondStereo set to STEREONONE and single bond directions set to NONE.
[04:32:15] Conflicting single bond directions around double bond at index 4.
[04:32:15]   BondStereo set to STEREONONE and single bond directions set to NONE.
[04:32:25] Conflicting single bond directions around double bond at index 4.
[04:32:25]   BondStereo set to STEREONONE and single bond directions set to NONE.
Processing SMILES:  91%|█████████ | 262304/288504 [01:30<00:47, 556.34it/s]   [04:37:18] Conflicting single bond directions around double bond at index 7.
[04:37:18]   BondStereo set to STEREONONE and single bond directions set to NONE.
Processing SMILES:  99%|█████████▉| 286822/288504 [02:

In [140]:
#print all unique values in the dictionary:
print(set(score_hits.values()))
print(set(score_inactives.values()))

{0, 2}
{0, 2, 5, 6}


In [141]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_13'):
    os.makedirs(f'{data_folder}/before_finished/step_13')

# save the scores:
with open(f'{data_folder}/before_finished/step_13/score_hits.json', 'w') as f:
    json.dump(score_hits, f)
with open(f'{data_folder}/before_finished/step_13/score_inactives.json', 'w') as f:
    json.dump(score_inactives, f)

In [142]:
#drop all compounds with a penalty score of 7:
to_drop_hits = []
to_drop_inactives = []
for cid, penalty_score in score_hits.items():
    if penalty_score == 7:
        to_drop_hits.append(cid)
for cid, penalty_score in score_inactives.items():
    if penalty_score == 7:
        to_drop_inactives.append(cid)

post13_hits = pre13_hits[~pre13_hits['PUBCHEM_CID'].isin(to_drop_hits)]
post13_inactives = pre13_inactives[~pre13_inactives['PUBCHEM_CID'].isin(to_drop_inactives)]

In [143]:
#save final_hits and inactives:
post13_hits.to_csv(f'{data_folder}/before_finished/step_13/post13_hits.csv', index=False)
post13_inactives.to_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', index=False)

# 14. Final handling of chemical representation

There are two problems that requires InChI update: 

(1) Some of the InChI will be missing, since the PubChem Identifier Exchange service might not able to find the corresponding InChI for the aromatized, neutralized SMILES. 

(2) While handling mixtures, some mixtures whose component molecules are identical will result in duplicates. Therefore, we need to check their activities. 
- If all duplicates share the same results (active/inactive), we keep one of them. 
- If duplicates of the same molecules returned different activity, we remove both of them. 

In [144]:
#import: 
pre14_hits = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_hits.csv', sep=',', header=0)
pre14_inactives = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', sep=',', header=0)

## 14.1 Update InChI

In [145]:
def smi_to_inchi(smi):
    mol = Chem.MolFromSmiles(smi)
    inchi = Chem.inchi.MolToInchi(mol)
    return inchi

In [146]:
count = 0 
for index, row in pre14_hits.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_hits.at[index, 'InChI'] = smi_to_inchi(row['PUBCHEM_EXT_DATASOURCE_SMILES'])
        count += 1
print(f'Updated {count} InChI values in pre14_hits')

count = 0
for index, row in pre14_inactives.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_inactives.at[index, 'InChI'] = smi_to_inchi(row['PUBCHEM_EXT_DATASOURCE_SMILES'])
        count += 1
print(f'Updated {count} InChI values in pre14_inactives')

[11:15:58] WARNING: Charges were rearranged



Updated 1 InChI values in pre14_hits


[11:16:00] WARNING: Omitted undefined stereo

[11:16:00] WARNING: Charges were rearranged

[11:16:01] WARNING: Omitted undefined stereo

[11:16:02] WARNING: Charges were rearranged; Omitted undefined stereo

[11:16:02] WARNING: Charges were rearranged

[11:16:02] WARNING: Charges were rearranged

[11:16:03] WARNING: Charges were rearranged; Omitted undefined stereo

[11:16:03] WARNING: Proton(s) added/removed

[11:16:03] WARNING: Proton(s) added/removed

[11:16:03] WARNING: Omitted undefined stereo

[11:16:03] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Proton(s) added/removed; Omitted undefined stereo

[11:16:04] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Proton(s) added/removed

[11:16:04] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Omitted undefined stereo

[11:16:04] WARNING: Omitted undefined stereo

[11:16:05] WARNING: Charges were rearranged

[11:16:05] W

Updated 66 InChI values in pre14_inactives


## 14.2 Handle duplicates

In [147]:
#Check if a mol in hit set appeared in inactive set:
for i in pre14_hits['PUBCHEM_EXT_DATASOURCE_SMILES']:
    if i in list(pre14_inactives['PUBCHEM_EXT_DATASOURCE_SMILES']):
        print(f'{i} SMILES appeared in both hit and inactive sets')
for i in pre14_hits['InChI']:
    if i in list(pre14_inactives['InChI']):
        print(f'{i} InChI appeared in both hit and inactive sets')

#Return all duplicates by comparing InChI:
final_hits_duplicates_InChI = pre14_hits[pre14_hits.duplicated(subset=['InChI'], keep=False)]
final_inactives_duplicates_InChI = pre14_inactives[pre14_inactives.duplicated(subset=['InChI'], keep=False)]
final_hits_duplicates_smi = pre14_hits[pre14_hits.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]
final_inactives_duplicates_smi = pre14_inactives[pre14_inactives.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]

print('Number of InChI duplicates in hits: ', len(final_hits_duplicates_InChI))
print('Number of InChI duplicates in inactives: ', len(final_inactives_duplicates_InChI))
print('Number of SMILES duplicates in hits: ', len(final_hits_duplicates_smi))
print('Number of SMILES duplicates in inactives: ', len(final_inactives_duplicates_smi))

Number of InChI duplicates in hits:  0
Number of InChI duplicates in inactives:  343
Number of SMILES duplicates in hits:  0
Number of SMILES duplicates in inactives:  341


In [148]:
if not os.path.exists(f'{data_folder}/before_finished/step_14'):
    os.makedirs(f'{data_folder}/before_finished/step_14')

#write all the duplicates to a file:
#write duplicates to a txt file: 
with open(f'{data_folder}/before_finished/step_14/duplicates.txt', 'w') as f:
    f.write('InChI duplicates in hits: \n')
    f.write(final_hits_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('InChI duplicates in inactives: \n')
    f.write(final_inactives_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in hits: \n')
    f.write(final_hits_duplicates_smi.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in inactives: \n')
    f.write(final_inactives_duplicates_smi.to_string())

In [149]:
#remove these duplicates, keep the first one: 
#by inchi:
final_hits = pre14_hits.drop_duplicates(subset=['InChI'], keep='first')
final_inactives = pre14_inactives.drop_duplicates(subset=['InChI'], keep='first')

In [150]:
if len(final_hits[final_hits.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]) == 0:
    print('No more duplicates in hits')

if len(final_inactives[final_inactives.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]) == 0:
    print('No more duplicates in inactives')

No more duplicates in hits
No more duplicates in inactives


In [151]:
# save: 
if not os.path.exists(f'{data_folder}/finished'):
    os.makedirs(f'{data_folder}/finished')
final_hits.to_csv(f'{data_folder}/finished/final_hits.csv',sep=',', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv',sep=',', index=False)

# 0.1. Import regression data: 

In [152]:
final_hits = pd.read_csv(f'{data_folder}/finished/final_hits.csv', sep=',', header=0)
final_inactives = pd.read_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', header=0)

In [153]:
#Data to be extracted from the assay:
regression_type = 'IC50'
col_list = ['PUBCHEM_CID', regression_type]
float_available_AIDs = [588401, 504840]

count = 0
for AID in float_available_AIDs:
    url = f'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={AID}'
    assay = pd.read_csv(url, usecols=col_list)

    #delete rows with nan values
    assay = assay.dropna(subset=col_list)
    
    #convert regression data to float
    assay[regression_type] = assay[regression_type].astype(float)

    #convert cids to int: 
    assay['PUBCHEM_CID'] = assay['PUBCHEM_CID'].astype(int)

    #reindex assay dataframe from 0:
    assay.reset_index(drop=True, inplace=True)

    #Create a new folder to save the data:
    if not os.path.exists(f'{data_folder}/before_finished/regression_data'):
        os.makedirs(f'{data_folder}/before_finished/regression_data')
    assay.to_csv(f'{data_folder}/before_finished/regression_data/AID{AID}_{regression_type}.csv', index=False)
    count += 1
    print(f'{count} out of {len(float_available_AIDs)} complete')

1 out of 2 complete
2 out of 2 complete


In [158]:
regression_CIDs = []
for AID in float_available_AIDs:
    exec(f'AID{AID}_regression = pd.read_csv("{data_folder}/before_finished/regression_data/AID{AID}_{regression_type}.csv", sep=",", header=0)')
    exec(f'regression_CIDs.extend(AID{AID}_regression["PUBCHEM_CID"].tolist())')
regression_CIDs = list(set(regression_CIDs))

In [161]:
#Make the regression data a dictionary: 
regression_dict1 = {}
for index, row in AID588401_regression.iterrows():
    regression_dict1[row['PUBCHEM_CID']] = row['IC50']

regression_dict2 = {}
for index, row in AID504840_regression.iterrows():
    regression_dict2[row['PUBCHEM_CID']] = row['IC50']

In [162]:
regression_dict = {}
for CID in regression_CIDs:
    if CID in regression_dict1 and CID in regression_dict2:
        regression_dict[CID] = (regression_dict1[CID] + regression_dict2[CID]) / 2
    elif CID in regression_dict1 and CID not in regression_dict2:
        regression_dict[CID] = regression_dict1[CID]
    elif CID not in regression_dict1 and CID in regression_dict2:
        regression_dict[CID] = regression_dict2[CID]

In [163]:
cut_off = "not reported" #note the cut off value for hits
inactive_fixed = 1000 #set a fixed value for inactives
unit = 'uM' #unit of the regression values
URLs = []
for AID in float_available_AIDs:
    URL = f'https://pubchem.ncbi.nlm.nih.gov/bioassay/{AID}'
    URLs.append(URL)

In [164]:
#Export the regression dict: 
with open(f'{data_folder}/before_finished/regression_data/regression_dict.json', 'w') as f:
    json.dump(regression_dict, f)

with open(f'{data_folder}/before_finished/regression_data/activity_value_details.txt', 'w') as f:
    f.write(f"""The regression data was extracted from AIDs: {float_available_AIDs}.
The {regression_type} values were averaged if there were multiple assays. 
The final {regression_type} data exported for {len(regression_dict)} unique compounds, parsable from column "activity_value" in the .csv files with unit {unit}.
The range of {regression_type} values reported were from {min(regression_dict.values())} to {max(regression_dict.values())}.
For final inactives, the {regression_type} value was set to {inactive_fixed} {unit} if dose-response info was not available. This value could be modified by users.
The cut-off for activity was {cut_off} uM, which was retrieved from the PubChem database from the following URL(s):
""")
    f.write(f'\n'.join(URLs))

In [165]:
for index, row in final_hits.iterrows():
    if row['PUBCHEM_CID'] in regression_dict:
        final_hits.at[index, 'activity_value'] = regression_dict[row['PUBCHEM_CID']]

for index, row in final_inactives.iterrows():
    if row['PUBCHEM_CID'] in regression_dict:
        final_inactives.at[index, 'activity_value'] = regression_dict[row['PUBCHEM_CID']]
    else: 
        final_inactives.at[index, 'activity_value'] = inactive_fixed

In [166]:
# Rename the columns:
final_hits = final_hits.rename(columns={
    'PUBCHEM_CID': 'CID', 
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})
final_inactives = final_inactives.rename(columns={
    'PUBCHEM_CID': 'CID', 
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})

In [167]:
#swap the positions of the columns InChI and activity_outcome: 
final_hits = final_hits[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
final_inactives = final_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [168]:
#export: 
final_hits.to_csv(f'{data_folder}/finished/final_hits.csv', sep=',', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', index=False)

# 0.2. Convert to txt for CORINA

In [169]:
# import the final hits and inactives:
final_hits = pd.read_csv(f'{data_folder}/finished/final_hits.csv', sep=',', header=0)
final_inactives = pd.read_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', header=0) 

# export to .txt files:
final_hits.to_csv(f'{data_folder}/finished/final_hits.txt', sep=';', index=False, header=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.txt', sep=';', index=False, header=False)

# 0.3. Exporting raw data without further curation (only smiles, inchi) for control experiments

In [176]:
for AID in AIDs: 
    exec(f"raw{AID} = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv', sep=',', header=0)")

#import inchi:
for AID in AIDs: 
    exec(f"std_inchi{AID} = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_{AID}.txt', sep='\t', header=None)")

#Update inchi
for AID in AIDs: 
    exec(f"""
raw_inchi_dict{AID} = dict(zip(std_inchi{AID}[0], std_inchi{AID}[1]))
raw{AID}['InChI'] = raw{AID}['PUBCHEM_CID'].map(raw_inchi_dict{AID})
""")

In [177]:
raw_hits = raw488975[raw488975['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
raw_inactives = raw488975[raw488975['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

In [179]:
# Get the average activity values of hits:
final_hits = pd.read_csv(f'{data_folder}/finished/final_hits.csv', sep=',', header=0)
activity_values = final_hits['activity_value'].tolist()
average_activity_value = sum(activity_values) / len(activity_values)

# Update the activity values: 
for index, row in raw_hits.iterrows():
    if row['PUBCHEM_CID'] in regression_dict:
        raw_hits.loc[index, 'activity_value'] = regression_dict[row['PUBCHEM_CID']]
    else:
        raw_hits.loc[index, 'activity_value'] = average_activity_value

for index, row in raw_inactives.iterrows():
    if row['PUBCHEM_CID'] in regression_dict:
        raw_inactives.loc[index, 'activity_value'] = regression_dict[row['PUBCHEM_CID']]
    else: 
        raw_inactives.loc[index, 'activity_value'] = inactive_fixed

In [181]:
# Add some other columns to match the format of the curated data: 
raw_hits.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_hits.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_hits.loc[:, 'small_organic_mol_from_mixture'] = np.nan

raw_inactives.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_organic_mol_from_mixture'] = np.nan

In [182]:
# Rename the columns:
raw_hits = raw_hits.rename(columns={
    'PUBCHEM_CID': 'CID', 
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})
raw_inactives = raw_inactives.rename(columns={
    'PUBCHEM_CID': 'CID', 
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})

#swap the positions of the columns InChI and activity_outcome: 
raw_hits = raw_hits[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
raw_inactives = raw_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [183]:
if not os.path.exists(f'{data_folder}/finished/control_data'):
    os.makedirs(f'{data_folder}/finished/control_data')

#save the hits and inactives
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.csv', sep=',', index=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.csv', sep=',', index=False)

#save as txt: 
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.txt', sep=';', index=False, header=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.txt', sep=';', index=False, header=False)